**TODO**
* get rid of requests we don't use
* annotate the code to explain what we're doing and why [tutorial!]
* have to explain what broad categories we wanted to analyze and why
    * clean up our doc about what categories we want to use and add it to the github, explain why we are only using certain [un] categories for now (most bang for our buck)

In [ ]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time


import matplotlib.pyplot as plt
import datetime
import re

In [ ]:
category_links_df = pd.read_csv('category-links.csv')
category_links_df

visited_categories = []
resolution_urls = list()
total_resolutions = 0

In [ ]:
def get_res_links_from_page(page_link, total_resolutions):
    
    links = []
    
    page = requests.get(page_link)
    if page.status_code != 200:
        raise Exception("Something went wrong loading category: " + category + ", error code: " + page.status_code)

    html = page.text
    soup = BeautifulSoup(html, 'html.parser')

    resolution_cnt = int(soup.find("strong", {"class": None}).text)

    if resolution_cnt > 50:
        if resolution_cnt%50 == 0:
            page_count = (int) (resolution_cnt/50)
        else: page_count = int((resolution_cnt/50) + 1)
    else: page_count = 1

    for page in range(page_count):

        for div in soup.find_all("div", {"class": "moreinfo"}):
            res_link_suffix = div.find("a")["href"]
            links.append('https://digitallibrary.un.org' + res_link_suffix)

        time.sleep(10)

        #load the next page if you're not already on the last page (0 indexing)
        if (page+1 != page_count):
            next_page_link_suffix = soup.find("span", {"class": "rec-navigation"}).findAll("a")[-1]["href"]
            next_page_link = 'https://digitallibrary.un.org' + next_page_link_suffix
            #print(page)
            #print(next_page_link)

            page = requests.get(next_page_link)
            if page.status_code != 200:
                raise Exception("Something went wrong loading next page, error code: " + page.status_code)

            html = page.text
            soup = BeautifulSoup(html, 'html.parser')
            
        
    if (total_resolutions + resolution_cnt != len(links)):
        raise Exception("resolution count does not match")
        
    return links

In [ ]:
#get the initial website with the categories
for cat, link in zip (category_link_df["un category"], category_link_df["link"]):
    if (cat not in visited_categories):
        try:
            cat_res_links = get_res_links_from_page(page_link, total_resolutions) #grab the links, may throw an exception
            resolution_urls.extend(cat_res_links) #append the new links list to the bigger old links list
            total_resolutions = len(resolution_urls) #update total resolution count for next runthrough
            visited_categories.append(cat)
        except:
            print("oh no the UN blocked you maybe :(")
        finally:
            time.sleep(3) #delay to hopefully prevent the un from detecting and blocking us

#write results to file
textfile = open("res_links.txt", "w")
for res_url in resolution_urls:
    textfile.write(res_url + "\n")
textfile.close()

In [ ]:
## get unique resolutions => THIS DOES NOT WORK????????????
# len(np.unique(resolution_urls))

#Make resolution_urls unique
def unique(list1):
    # initialize a null list
    unique_list = []
     
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return unique_list

# ex of function use in next line
#unq_res_urls = unique(resolution_urls)

In [ ]:
#Get vote data from each resolution

#resolution df (x= index, y = name, year)
#country_votes df (row = country, col= resolution indices) row,col = vote

resolutions
for res_url in unq_res_urls:
    res_page = requests.get(res_url)
    if res_page.status_code != 200:
        raise Exception("Something went wrong loading resolution, error code: " + res_page.status_code)
        
    html = res_page.text
    soup = BeautifulSoup(html, 'html.parser')

    metadata = soup.find_all("span", {"class": "value col-xs-12 col-sm-9 col-md-10"})

    #number of metadata sections changes from resolution to resolution, so start counting from bottom for some
    res_title = metadata[0] 
    #res_title.text
    vote_summary = metadata[len(metadata) -4]
    vote_date = metadata[len(metadata) -3]
    vote = metadata[len(metadata) -2]
    time.sleep(10) #can change later
    
    #STORE THIS INFO IN DF EACH TIME